In [1]:
from tensorflow.keras.models import Model
import tensorflow as tf
from model import lstm_vit
from helpers import f1
import warnings
import numpy as np
import matplotlib.pyplot as plt
from data_loader import data_generator
warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    
    tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow.keras import backend as K

K.clear_session()

2025-08-05 11:41:17.809076: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-05 11:41:17.847540: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-05 11:41:17.847593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-05 11:41:17.848954: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-05 11:41:17.855783: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model = lstm_vit()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1])
model.summary()

2025-08-05 11:41:21.979019: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46672 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:21:00.0, compute capability: 8.9


Model: "LSTM_ViT"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 15, 256, 256, 3   0         
                             )]                                  
                                                                 
 temporal_encoding (TimeDis  (None, 15, 16, 16, 256)   2871776   
 tributed)                                                       
                                                                 
 spatial_pooling (TimeDistr  (None, 15, 256)           0         
 ibuted)                                                         
                                                                 
 temporal_lstm (Bidirection  (None, 15, 128)           164352    
 al)                                                             
                                                                 
 spatial_projection (TimeDi  (None, 15, 65536)         845

In [3]:
train_img_path = 'data_v3_processed/train/images/images'
train_masks_path = 'data_v3_processed/train/masks/masks'

val_img_path = 'data_v3_processed/val/images/images'
val_masks_path = 'data_v3_processed/val/masks/masks'

train_gen = data_generator(train_img_path, train_masks_path, 32)
val_gen = data_generator(val_img_path, val_masks_path, 32)

In [4]:
history = model.fit(train_gen,
                    epochs=1,
                    batch_size=32,
                    steps_per_epoch=6000)

Building sequence list and filtering empty sequences...
Total valid sequences: 220
Skipped empty sequences: 270
Reverse time order: True


2025-08-05 11:41:40.399721: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-08-05 11:41:47.628999: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f533568cdf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-05 11:41:47.629046: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2025-08-05 11:41:47.633858: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1754394107.755599    2978 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6000/6000 [==============================] - 7706s 1s/step - loss: 0.0176 - f1: 0.3163


In [5]:
model.save("lstm_vit_model.h5")